In [25]:
from datasets import load_dataset

EXIST_train = load_dataset("csv", data_files="EXIST2021_training.tsv", sep="\t")['train']
EXIST_test = load_dataset("csv", data_files="EXIST2021_test_labeled.tsv", sep="\t")['train']

EXIST_train = EXIST_train.remove_columns(['test_case', 'id', 'source', 'language'])
EXIST_test = EXIST_test.remove_columns(['test_case', 'id', 'source', 'language'])

EXIST_train = EXIST_train.rename_column('task1','label_sexist')
EXIST_test = EXIST_test.rename_column('task1','label_sexist')

EXIST_train = EXIST_train.class_encode_column("label_sexist")
EXIST_test = EXIST_test.class_encode_column("label_sexist")



Casting to class labels:   0%|          | 0/6977 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4368 [00:00<?, ? examples/s]

In [28]:
EXIST_test[0]

{'text': "Pennsylvania State Rep horrifies with opening prayermooslime politican Movita Johnson-Harrell offended by the word 'Jesus' and 'God'.In a mainly Christian country praying to God/Jesus - what a scandal /sarc. It's the well-known mooslime strategy: playing the victim card and make demands.https://truepundit.com/pennsylvania-state-rep-horrifies-democrats-with-opening-prayer-at-the-name-of-jesus-every-knee-will-bow/",
 'label_sexist': 0,
 'task2': 'non-sexist'}

In [30]:
from datasets import ClassLabel

new_features = EXIST_train.features.copy()
new_features['label_sexist'] = ClassLabel(names=["not sexist", "sexist"])
EXIST_train = EXIST_train.cast(new_features)
EXIST_test = EXIST_train.cast(new_features)

## Split the dataset

split_dataset = EXIST_train.train_test_split(0.2,stratify_by_column='label_sexist')
train_dataset = split_dataset['train']
validation_dataset = split_dataset['test']
test_dataset = EXIST_test

## combine the dataset

from datasets import DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})
print(dataset_dict)

Casting the dataset:   0%|          | 0/6977 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label_sexist', 'task2'],
        num_rows: 5581
    })
    validation: Dataset({
        features: ['text', 'label_sexist', 'task2'],
        num_rows: 1396
    })
    test: Dataset({
        features: ['text', 'label_sexist', 'task2'],
        num_rows: 6977
    })
})


In [31]:



## Upload the dataset

import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/EXIST2021")

Token read successfully
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/yzheng/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]